# Strategy Rules

- Determine Upward Trend: Identify an upward trend by checking if the closing price is above a moving average (e.g., EMA).
- Check for Bearish Volatility: Ensure there is no bearish volatility by analyzing the ATR indicators.
- Entry Signal: Generate an entry signal if there is an upward trend and no bearish volatility. Exit Signal: Implement a trailing stop-loss to generate an exit signal and protect profits.
- Exit Signal: Implement a trailing stop-loss to generate an exit signal and protect profits.
- tight trailing stop when if intrade and bearish volatility.


In [47]:
from momentum_strategy_ready import SetUpBotKucoin

In [35]:
from momentum_strategy_ready import SetUpBotKucoin
import pandas as pd
import numpy as np
import vectorbt as vbt
from datetime import datetime
import os 
from numba import njit
from vectorbt.portfolio.enums import Direction, SizeType
import talib
import requests
import time


symbol = "BTC-USDT"
timeframe = "1day"
market_type = "spot"
start_time = "2021-01-01 00:00:00"
end_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
# end_time = "2021-06-10 00:00:00"

skip_fetch = False


In [36]:

if not skip_fetch:
    print("Fetching data...")
    def fetch_kucoin_candles_chunk(symbol, market_type="spot", timeframe="1min", start_time=None, end_time=None):
        time.sleep(0.2)  # Rate limiting
        base_url = "https://api.kucoin.com" if market_type.lower() == "spot" else "https://api-futures.kucoin.com"
        url = base_url + "/api/v1/market/candles"
        params = {"type": timeframe, "symbol": symbol.upper()}
        if start_time:
            params["startAt"] = int(time.mktime(time.strptime(start_time, "%Y-%m-%d %H:%M:%S")))
        if end_time:
            params["endAt"] = int(time.mktime(time.strptime(end_time, "%Y-%m-%d %H:%M:%S")))
        try:
            resp = requests.get(url, params=params, timeout=10)
            resp.raise_for_status()
            data = resp.json()
            if data.get("code") == "200000":
                return data["data"]
            else:
                raise Exception(f"KuCoin API error: {data}")
        except requests.exceptions.RequestException as e:
            raise Exception(f"Request failed: {str(e)}")

    def fetch_all_kucoin_candles(symbol, market_type="spot", timeframe="1min", start_time=None, end_time=None):
        chunks = []
        current_end = end_time
        start_timestamp = int(time.mktime(time.strptime(start_time, "%Y-%m-%d %H:%M:%S")))
        
        while True:
            try:
                chunk = fetch_kucoin_candles_chunk(symbol, market_type, timeframe, start_time, current_end)
                if not chunk:
                    print("No more data available")
                    break
                earliest_ts = int(chunk[-1][0])
                print(f"Fetched {len(chunk)} candles from {datetime.fromtimestamp(earliest_ts)}")
                chunks.extend(chunk)
                if earliest_ts <= start_timestamp:
                    print("Reached start time")
                    break
                current_end = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(earliest_ts - 60))
            except Exception as e:
                print(f"Error occurred: {str(e)}")
                break
        
        if not chunks:
            return []
        
        chunks.sort(key=lambda x: x[0])  # Sort by timestamp
        result = [candle for candle in chunks if start_timestamp <= int(candle[0]) <= int(time.mktime(time.strptime(end_time, "%Y-%m-%d %H:%M:%S")))]
        print(f"Total candles fetched: {len(result)}")
        return result


    data = fetch_all_kucoin_candles(symbol, market_type, timeframe, start_time, end_time)
    print(f"Final data length: {len(data)}")
    data
else:
    print("Skipping fetch...")

Fetching data...
Fetched 1466 candles from 2021-01-01 00:00:00
Reached start time
Total candles fetched: 1466
Final data length: 1466


In [37]:

# Assuming 'data' is the list of candlestick data
df = pd.DataFrame(data)
df[['timestamp', 'open', 'close', 'high', 'low']] = df[[0, 1, 2, 3, 4]]
df.drop([0, 1, 2, 3, 4, 5, 6], axis=1, inplace=True)

# Explicitly cast 'timestamp' to numeric type
df['timestamp'] = pd.to_numeric(df['timestamp'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', utc=True)

df.set_index('timestamp', inplace=True)
df.sort_index(inplace=True)
df[['open', 'close', 'high', 'low']] = df[['open', 'close', 'high', 'low']].astype(float)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1466 entries, 2021-01-01 00:00:00+00:00 to 2025-01-05 00:00:00+00:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    1466 non-null   float64
 1   close   1466 non-null   float64
 2   high    1466 non-null   float64
 3   low     1466 non-null   float64
dtypes: float64(4)
memory usage: 57.3 KB


,open,close,high,low
timestamp,,,,
2021-01-01 00:00:00+00:00,28924.9,29333.2,29607.1,28633.0
2021-01-02 00:00:00+00:00,29333.2,32185.4,33286.8,28949.2
2021-01-03 00:00:00+00:00,32181.3,33000.4,34766.0,31500.0
2021-01-04 00:00:00+00:00,33005.9,31993.0,33600.2,27500.0
2021-01-05 00:00:00+00:00,31993.1,33938.6,34371.0,29806.1
...,...,...,...,...
2025-01-01 00:00:00+00:00,93574.4,94584.1,95150.0,92900.8
2025-01-02 00:00:00+00:00,94584.2,96983.8,97839.1,94380.5
2025-01-03 00:00:00+00:00,96983.9,98173.9,98976.9,96105.4


In [74]:
strategy = SetUpBotKucoin("BTC-USDT", '1day')
strategy.df = df.copy()

strategy.calculate_indicators(atr_length_sl=5, atr_length_vola=5, ema_length=150)
strategy.get_signal(lookback_high=7, atr_vol_multiplier=1.5)

for i in range(1, len(strategy.df)):
    # First check if we got stopped out
    strategy.check_sl(i)
    
    # Only proceed with trade management if we're not stopped out
    if not strategy.df['sl_hit'].iloc[i]:
        strategy.handle_signal(i)
        strategy.update_trail_sl(i)
        strategy.set_in_trade(i)

strategy.df

,open,close,high,low,atr_sl,atr_vola,ema_trend,r_high,is_bearish_vola,is_bullish,signal,in_trade,trail_sl,sl_hit,update_tsl,trail_source
timestamp,,,,,,,,,,,,,,,,
2021-01-01 00:00:00+00:00,28924.9,29333.2,29607.1,28633.0,NaN,NaN,NaN,NaN,False,False,None,0,NaN,False,False,NaN
2021-01-02 00:00:00+00:00,29333.2,32185.4,33286.8,28949.2,NaN,NaN,NaN,NaN,False,False,caution,0,NaN,False,False,NaN
2021-01-03 00:00:00+00:00,32181.3,33000.4,34766.0,31500.0,NaN,NaN,NaN,NaN,False,False,caution,0,NaN,False,False,NaN
2021-01-04 00:00:00+00:00,33005.9,31993.0,33600.2,27500.0,NaN,NaN,NaN,NaN,False,False,caution,0,NaN,False,False,NaN
2021-01-05 00:00:00+00:00,31993.1,33938.6,34371.0,29806.1,NaN,NaN,NaN,NaN,False,False,caution,0,NaN,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01 00:00:00+00:00,93574.4,94584.1,95150.0,92900.8,3322.378640,3322.378640,80316.082328,99970.8,True,True,caution,0,NaN,False,False,95156.0
2025-01-02 00:00:00+00:00,94584.2,96983.8,97839.1,94380.5,3349.622912,3349.622912,80536.846800,97839.1,False,True,caution,0,NaN,False,False,94380.5
2025-01-03 00:00:00+00:00,96983.9,98173.9,98976.9,96105.4,3253.998330,3253.998330,80770.450154,98976.9,False,True,buy,1,92851.401670,False,False,96105.4


In [75]:
import plotly.graph_objects as go
dft = strategy.df

fig = go.Figure()

fig.add_trace(go.Candlestick(x=dft.index,open = df['open'],close = dft['close'], low= dft['low'],high= dft['high'],name='candlesticks'))

fig.add_trace(go.Scatter(x=dft.index, y=dft['ema_trend'], mode='lines', name='ema', line=dict(color='blue')))

buy = dft[dft['signal'] == 'buy']
caution = df[dft['signal'] == 'caution']

fig.add_trace(go.Scatter(x=dft.index, y=[0]*len(caution), mode='markers', name='caution', line=dict(color='yellow')))
fig.add_trace(go.Scatter(x=buy.index, y=[0]*len(buy), mode='markers', name='buy', line=dict(color='green')))


fig.update_layout(title='Chart',
                  xaxis_title = 'date',
                  yaxis_title='price',
                  xaxis_rangeslider_visible=False,
                  hovermode='x unified')

fig.show()

## creating indicator to set signals long ONLY for simplicity

In [38]:
import pandas as pd
import numpy as np
import vectorbt as vbt

skip_cell = False

if not skip_cell:
    print("This cell is executed.")

    def momentum_strategy(close, high, low, ema_length):
        # Calculate EMA
        EMA = vbt.IndicatorFactory.from_talib('EMA')
        ema = EMA.run(close, timeperiod=ema_length).real.to_numpy()

        # Calculate conditions
        is_bullish = close > ema
        is_caution = close < ema
        signal_buy = is_bullish & ~is_caution

        conditions = [
            signal_buy,
            is_caution
        ]

        values = [10, 5]
        
        # Use np.select to create final signal
        signal = np.select(conditions, values, default=0)
        
        # Ensure signal has the same shape as input
        return signal.reshape(close.shape)

    # Create the indicator factory
    momentum_indicator = vbt.IndicatorFactory(
        class_name='MomentumStrategy',
        short_name='momentum',
        input_names=['Close', 'High', 'Low'],
        param_names=['ema_length'],
        output_names=['signal']
    ).from_apply_func(momentum_strategy)

    # Define parameter values
    ema_length = np.arange(10, 290, 5)

    # Run the indicator
    signal = momentum_indicator.run(
        df['close'], df['high'], df['low'],
        ema_length=ema_length,
        param_product=True  
    )
    print(signal.signal.shape)
    df_indicator = signal.signal

else:
    print("This cell is skipped.")

This cell is executed.
(1466, 56)


In [39]:
df_indicator

momentum_ema_length,10,15,20,25,30,35,40,45,50,55,...,240,245,250,255,260,265,270,275,280,285
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00+00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-01-02 00:00:00+00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-01-03 00:00:00+00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-01-04 00:00:00+00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-01-05 00:00:00+00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01 00:00:00+00:00,5,5,5,5,5,5,5,10,10,10,...,10,10,10,10,10,10,10,10,10,10
2025-01-02 00:00:00+00:00,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
2025-01-03 00:00:00+00:00,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10


In [40]:
def repeat_series_horizontally(series, target_shape):
    data = series.values.reshape(-1, 1)  # Convert to column vector
    return np.repeat(data, target_shape[1], axis=1)

nedded_copies = len(df_indicator.columns)
print('needed columns: ',nedded_copies)
print('needed rows: ',len(df_indicator))
print('shape:', df_indicator.shape)
print()

# copying the close price to the shape of the indicator
print('original shape:\n\n', df['close'])
dublicate_close = repeat_series_horizontally(df['close'], df_indicator.shape)
df_dublicate_close  = pd.DataFrame(dublicate_close,df.index)
df_dublicate_close 



needed columns:  56
needed rows:  1466
shape: (1466, 56)

original shape:

 timestamp
2021-01-01 00:00:00+00:00    29333.2
2021-01-02 00:00:00+00:00    32185.4
2021-01-03 00:00:00+00:00    33000.4
2021-01-04 00:00:00+00:00    31993.0
2021-01-05 00:00:00+00:00    33938.6
                              ...   
2025-01-01 00:00:00+00:00    94584.1
2025-01-02 00:00:00+00:00    96983.8
2025-01-03 00:00:00+00:00    98173.9
2025-01-04 00:00:00+00:00    98215.4
2025-01-05 00:00:00+00:00    97900.4
Name: close, Length: 1466, dtype: float64


,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00+00:00,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,...,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2
2021-01-02 00:00:00+00:00,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,...,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4
2021-01-03 00:00:00+00:00,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,...,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4
2021-01-04 00:00:00+00:00,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,...,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0
2021-01-05 00:00:00+00:00,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,...,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01 00:00:00+00:00,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,...,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1
2025-01-02 00:00:00+00:00,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,...,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8
2025-01-03 00:00:00+00:00,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,...,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9


## compare the shape of the copied closing prices 

In [41]:
df_custom_indiator = df_indicator

if df_custom_indiator.shape == df_dublicate_close.shape:
    df_renamed_close = df_dublicate_close
    df_renamed_close.columns = df_custom_indiator.columns
    print("Columns have been renamed ")
else:
    print("The DataFrames do not have the same shape.")

# Display the resulting DataFrame
df_renamed_close.head()
df_renamed_close

Columns have been renamed 


momentum_ema_length,10,15,20,25,30,35,40,45,50,55,...,240,245,250,255,260,265,270,275,280,285
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00+00:00,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,...,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2
2021-01-02 00:00:00+00:00,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,...,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4
2021-01-03 00:00:00+00:00,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,...,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4
2021-01-04 00:00:00+00:00,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,...,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0
2021-01-05 00:00:00+00:00,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,...,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01 00:00:00+00:00,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,...,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1,94584.1
2025-01-02 00:00:00+00:00,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,...,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8
2025-01-03 00:00:00+00:00,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,...,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9


In [42]:
atr_values = talib.ATR(df['high'], df['low'], df['close'], timeperiod=7).to_numpy().flatten()
print(type(atr_values))
print(atr_values.shape)

<class 'numpy.ndarray'>
(1466,)


In [46]:
# Numba-compiled order function
@njit
def order_func_nb(c, high, low, open_, entries, sl_prices, tp_prices,entry_price,atr_values):
    close_price = c.close[c.i, c.col]
    # print("INDEX", c.i)
    # print("COL", c.col)
    # print("high", high[c.i])
    # print('open', open_[c.i])
    # print("close", close_price)
    # print('low', low[c.i])
    # print('position size :', c.position_now)
    # print('cash:', c.cash_now)
    # print('entries:', entries[c.i, c.col])
    # print('sl_prices:', sl_prices[c.i])
    # print('tp_prices:', tp_prices[c.i])
    # print('entry_price:', entry_price[c.i])
    # print()
    #calculate caution 

# if in position 
    if (c.position_now > 0):
        # Check if SL is hit
        # if c.close[c.i-1, c.col] <= sl_prices[c.i]:
        if open_[c.i] <= sl_prices[c.i]:
        # if low[c.i] <= sl_prices[c.i]:
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,  # Close position
                price=open_[c.i],  # Current closing price
                size_type=SizeType.Amount,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002)
            # print('sl hit at index',c.i)
            # print('sl order', value)

            return value

        # sl update for long position


        if (entries[c.i-1, c.col] == 5):  
            highest_low = np.max(low[c.i-7:c.i])
            update2 = highest_low - atr_values[c.i] * 0.2
            if update2 > sl_prices[c.i]:
                # print('sl update before',c.i,sl_prices[c.i])
                sl_prices[:]= update2

        #losen sl for less valatility 
        if (entries[c.i-1, c.col] == 10): 
            highest_low = np.max(low[c.i-7:c.i])
            update = highest_low - atr_values[c.i]
            if update > sl_prices[c.i]:
                # print('sl update before',c.i,sl_prices[c.i])
                sl_prices[:]= update
  


    # if not in position search for position to enter
    elif (c.position_now == 0) & (c.i != 0):
        if entries[c.i-1, c.col] == 10 :
            sl_prices[:] = np.nan
            entry_price[:] = open_[c.i]
            order = vbt.portfolio.nb.order_nb(
                size=1,  # Adjusted order size
                price=open_[c.i],  # Current closing price
                size_type=SizeType.Percent,  # Specify size type
                direction=Direction.LongOnly,  # Long-only trading
                fees=0.001,  # No fees
                slippage=0.002,  # No slippage
                allow_partial=False,  # Do not allow partial fills
                raise_reject=True  # Raise an error if the order is rejected
            )

            sl_prices[:] = low[c.i] - atr_values[c.i] 
            # print('entry', order)
            return order

    
    return vbt.portfolio.enums.NoOrder


close = df_renamed_close
entries = df_custom_indiator.to_numpy()

open_ = df['open'].to_numpy().flatten()
high = df['high'].to_numpy().flatten()
low = df['low'].to_numpy().flatten()

atr_values = talib.ATR(df['high'], df['low'], df['close'], timeperiod=7).to_numpy().flatten()

# Create an array to store SL prices
sl_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
tp_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
entry_price = np.full(close.shape[0], np.nan)  # Use a 1D array

# Create portfolio
pf = vbt.Portfolio.from_order_func(
    close,           # Price DataFrame
    order_func_nb,
    high,
    low,
    open_,
    entries,    # Order function
    sl_prices,
    tp_prices,
    entry_price,
    atr_values,  # Pass the SL prices array
    init_cash=500  # Initial cash balance
)

# Display some portfolio performance metrics
print("Total Return:", pf.total_return())
print("\nOrder Records:")

pf.orders.records_readable




ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [44]:
total_return = pf.total_return()
max_dd = pf.max_drawdown()
return_and_maxdd = pd.concat([total_return, max_dd], axis=1)
return_and_maxdd.sort_values(by='total_return', ascending=False).head(50)



,total_return,max_drawdown
momentum_ema_length,,
35,1.457213,-0.463576
30,1.283120,-0.489672
115,1.054258,-0.448581
140,0.928896,-0.478199
40,0.928340,-0.483423
135,0.910425,-0.474198
15,0.810071,-0.592990
130,0.784053,-0.474198
125,0.757966,-0.474198


In [45]:
# Access the setup for a specific name, e.g., '(4, 100, 1.0, 4)'
setup_name = (8, 160, 1.8, 7)
specific_pf = pf[setup_name]
specific_pf.plot().show()



KeyError: 'key of type tuple not found and not a MultiIndex'

In [26]:
specific_pf2 = pf.select_one(setup_name)
orders = specific_pf2.orders.records_readable
orders

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,109,"(8, 160, 1.8, 7)",2021-07-31 00:00:00+00:00,0.011812,42287.3058,0.499500,Buy
1,110,"(8, 160, 1.8, 7)",2021-08-03 00:00:00+00:00,0.011812,39069.5044,0.461492,Sell
2,111,"(8, 160, 1.8, 7)",2021-08-07 00:00:00+00:00,0.010730,42925.1790,0.460570,Buy
3,112,"(8, 160, 1.8, 7)",2021-08-31 00:00:00+00:00,0.010730,46887.0380,0.503079,Sell
4,113,"(8, 160, 1.8, 7)",2021-09-01 00:00:00+00:00,0.010639,47189.6910,0.502074,Buy
...,...,...,...,...,...,...,...
96,205,"(8, 160, 1.8, 7)",2024-12-12 00:00:00+00:00,0.012320,101319.1338,1.248271,Buy
97,206,"(8, 160, 1.8, 7)",2024-12-19 00:00:00+00:00,0.012320,100002.4942,1.232050,Sell
98,207,"(8, 160, 1.8, 7)",2024-12-26 00:00:00+00:00,0.012341,99633.7698,1.229588,Buy
99,208,"(8, 160, 1.8, 7)",2024-12-30 00:00:00+00:00,0.012341,93552.3204,1.154536,Sell


In [28]:
orders_time_filered = orders[orders['Timestamp'] > '2023-01-01 00:00:00']
orders_time_filered

,Order Id,Column,Timestamp,Size,Price,Fees,Side
20,129,"(8, 160, 1.8, 7)",2023-01-16 00:00:00+00:00,0.025613,20916.0486,0.535714,Buy
21,130,"(8, 160, 1.8, 7)",2023-02-07 00:00:00+00:00,0.025613,22714.4800,0.581777,Sell
22,131,"(8, 160, 1.8, 7)",2023-02-17 00:00:00+00:00,0.024640,23563.6332,0.580614,Buy
23,132,"(8, 160, 1.8, 7)",2023-02-25 00:00:00+00:00,0.024640,23140.7258,0.570194,Sell
24,133,"(8, 160, 1.8, 7)",2023-03-02 00:00:00+00:00,0.024034,23677.1598,0.569054,Buy
...,...,...,...,...,...,...,...
96,205,"(8, 160, 1.8, 7)",2024-12-12 00:00:00+00:00,0.012320,101319.1338,1.248271,Buy
97,206,"(8, 160, 1.8, 7)",2024-12-19 00:00:00+00:00,0.012320,100002.4942,1.232050,Sell
98,207,"(8, 160, 1.8, 7)",2024-12-26 00:00:00+00:00,0.012341,99633.7698,1.229588,Buy
99,208,"(8, 160, 1.8, 7)",2024-12-30 00:00:00+00:00,0.012341,93552.3204,1.154536,Sell
